####Fine-tuning en-indic model of indicTrans:

In [ ]:
!pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu117
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 960.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 89.8 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0+cu118
    Uninstalling torch-2.0.0+cu118:
      Successfully uninstalled torch-2.0.0+cu118
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.15.1+cu118
    Uninstalling torchvision-0.15.1+cu118:
      Successfully uninstalled torchvision-0.15.1+cu118
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.0.1+cu118
    Uninstalling torchaudio-2.0.1+cu118:
      Successfully uninstalled torchaudio-2.0.1+cu118
ERROR: pip's dependency resolver does not currently take into account all th

In [ ]:
#Making a separate directory for keeping finetuning data and model

!mkdir /content/finetuning

In [ ]:
%cd /content/finetuning

/content/finetuning


In [ ]:
# clone the repo for running evaluation
!git clone https://github.com/AI4Bharat/indicTrans.git
%cd indicTrans
# clone requirements repositories
!git clone https://github.com/anoopkunchukuttan/indic_nlp_library.git
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
!git clone https://github.com/rsennrich/subword-nmt.git
%cd ..

Cloning into 'indicTrans'...
remote: Enumerating objects: 697, done.
remote: Counting objects: 100% (400/400), done.
remote: Compressing objects: 100% (158/158), done.
remote: Total 697 (delta 278), reused 344 (delta 240), pack-reused 297
Receiving objects: 100% (697/697), 2.64 MiB | 10.89 MiB/s, done.
Resolving deltas: 100% (405/405), done.
/content/finetuning/indicTrans
Cloning into 'indic_nlp_library'...
remote: Enumerating objects: 1362, done.
remote: Counting objects: 100% (143/143), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 1362 (delta 111), reused 98 (delta 93), pack-reused 1219
Receiving objects: 100% (1362/1362), 9.56 MiB | 15.48 MiB/s, done.
Resolving deltas: 100% (721/721), done.
Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 139 (delta 2), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (139/139), 

In [ ]:
! sudo apt install tree
# Install the necessary libraries
!pip install sacremoses pandas mock sacrebleu tensorboardX pyarrow indic-nlp-library
! pip install mosestokenizer subword-nmt
# Install fairseq from source
!git clone https://github.com/pytorch/fairseq.git
%cd fairseq
# !git checkout da9eaba12d82b9bfc1442f0e2c6fc1b895f4d35d
!pip install ./
! pip install xformers
%cd ..

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 43.0 kB of archives.
After this operation, 115 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 tree amd64 1.8.0-1 [43.0 kB]
Fetched 43.0 kB in 1s (84.3 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package tree.
(Reading database ... 122518 files and directories currently install

In [ ]:
# add fairseq folder to python path
import os
os.environ['PYTHONPATH'] += ":/content/fairseq/"
from fairseq import checkpoint_utils, distributed_utils, options, tasks, utils

In [ ]:
# downloading the en-indic model
!wget https://ai4b-public-nlu-nlg.objectstore.e2enetworks.net/indic2en.zip
!unzip indic2en.zip

--2023-05-04 07:17:34--  https://ai4b-public-nlu-nlg.objectstore.e2enetworks.net/indic2en.zip
Resolving ai4b-public-nlu-nlg.objectstore.e2enetworks.net (ai4b-public-nlu-nlg.objectstore.e2enetworks.net)... 101.53.136.19, 164.52.210.97, 101.53.152.30, ...
Connecting to ai4b-public-nlu-nlg.objectstore.e2enetworks.net (ai4b-public-nlu-nlg.objectstore.e2enetworks.net)|101.53.136.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4759117228 (4.4G) [application/zip]
Saving to: ‘indic2en.zip’

indic2en.zip        100%[===================>]   4.43G  11.2MB/s    in 6m 48s  

2023-05-04 07:24:24 (11.1 MB/s) - ‘indic2en.zip’ saved [4759117228/4759117228]

Archive:  indic2en.zip
   creating: indic-en/
   creating: indic-en/vocab/
  inflating: indic-en/vocab/bpe_codes.32k.SRC  
  inflating: indic-en/vocab/vocab.SRC  
  inflating: indic-en/vocab/vocab.TGT  
  inflating: indic-en/vocab/bpe_codes.32k.TGT  
   creating: indic-en/final_bin/
  inflating: indic-en/final_bin/prepro

In [ ]:
%cd /content/finetuning
!wget "https://drive.google.com/uc?id=19rn5bsCi_VU8aR4Yl0FYu-Y3f06_L7tg&export=download" -O dataset.zip
!unzip dataset.zip
%cd /content/finetuning/indicTrans

/content/finetuning
--2023-05-04 07:25:39--  https://drive.google.com/uc?id=19rn5bsCi_VU8aR4Yl0FYu-Y3f06_L7tg&export=download
Resolving drive.google.com (drive.google.com)... 142.250.107.113, 142.250.107.138, 142.250.107.101, ...
Connecting to drive.google.com (drive.google.com)|142.250.107.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-10-a0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/sj8bskmsa9iq6ad72uls3pk48cln28kv/1683185100000/10066636681463159322/*/19rn5bsCi_VU8aR4Yl0FYu-Y3f06_L7tg?e=download&uuid=073b67ec-8aa1-4bd2-b9b8-2d313f0b7d00 [following]
--2023-05-04 07:25:41--  https://doc-10-a0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/sj8bskmsa9iq6ad72uls3pk48cln28kv/1683185100000/10066636681463159322/*/19rn5bsCi_VU8aR4Yl0FYu-Y3f06_L7tg?e=download&uuid=073b67ec-8aa1-4bd2-b9b8-2d313f0b7d00
Resolving doc-10-a0-docs.googleusercontent.com (doc-10-a0-docs.googleusercontent.com).

In [ ]:
%%shell
# Creating working directory and copying data into it:
# exp_dir: Directory in which preprocessed data is stored
# download_dir: Directory in which extracted en-indic model is stored
# Copy the vocab from downloaded model to working directory

exp_dir=/content/finetuning/dataset_finetune
download_dir=/content/finetuning/indic-en
src_lang=mr
tgt_lang=en
cp -r $download_dir/vocab $exp_dir

echo "Running experiment ${exp_dir} on ${src_lang} to ${tgt_lang}"

# Preparing directory structure:
# train_processed_dir, devtest_processed_dir: For processed data
# train_norm_dir, devtest_norm_dir: For Normalization output

train_processed_dir=$exp_dir/data
devtest_processed_dir=$exp_dir/data

mkdir -p $train_processed_dir
mkdir -p $devtest_processed_dir

train_norm_dir=$exp_dir/norm/$src_lang-$tgt_lang
devtest_norm_dir=$exp_dir/norm/$src_lang-$tgt_lang

mkdir -p $train_norm_dir
mkdir -p $devtest_norm_dir


# Pre-processing of train, dev and test set

datasets=(train dev test)

for dataset in ${datasets[@]};do
		if [ $dataset == train ]; then
			in_dir=$exp_dir/$dataset/$src_lang-$tgt_lang
		else
			in_dir=$exp_dir/$dataset
		fi

		op_dir=$exp_dir/norm/$src_lang-$tgt_lang

    infname_src=$in_dir/$dataset.$src_lang
    infname_tgt=$in_dir/$dataset.$tgt_lang

		outfname_src=$op_dir/$dataset.$src_lang
    outfname_tgt=$op_dir/$dataset.$tgt_lang

    echo "Applying normalization and script conversion for $dataset $lang"

    input_size=`python scripts/preprocess_translate.py $infname_src $outfname_src $src_lang true`
    input_size=`python scripts/preprocess_translate.py $infname_tgt $outfname_tgt $tgt_lang true`

    echo "Number of sentences in $dataset $lang: $input_size"
done

#Concatenate Data
python scripts/concat_joint_data.py $exp_dir/norm $exp_dir/data $src_lang $tgt_lang 'train'
python scripts/concat_joint_data.py $exp_dir/norm $exp_dir/data $src_lang $tgt_lang 'dev'
python scripts/concat_joint_data.py $exp_dir/norm $exp_dir/data $src_lang $tgt_lang 'test'

#Apply Byte Pair Encoding (BPE):
#BPE is a form of data compression algorithm in which the most common pair of consecutive bytes of data is replaced by the by the byte that has not occured in that data.
echo "Applying bpe to the new finetuning data"
bash apply_single_bpe_traindevtest_notag.sh $exp_dir

#Create Final Prepared Data Dir from which Binarizer will take input, this dir will contain data with special language tags
bin_input_data_dir=$exp_dir/final
mkdir -p $bin_input_data_dir

# Adding special language tags to indicate the source and the target language
#Hence, to translate from English to Marathi, give input as _src_en_ _tgt_mr_

echo "Adding language tags"
python scripts/add_joint_tags_translate.py $exp_dir 'train'
python scripts/add_joint_tags_translate.py $exp_dir 'dev'
python scripts/add_joint_tags_translate.py $exp_dir 'test'

#Preparing directory to store binarized output
bin_out_data_dir=$exp_dir/final_bin

rm -rf $bin_out_data_dir

#To get number of cores processing simultaneously
num_workers=`python -c "import multiprocessing; print(multiprocessing.cpu_count())"`

#Binarization of data
echo "Binarizing data"
fairseq-preprocess --source-lang SRC --target-lang TGT \
 --trainpref $bin_input_data_dir/train --validpref $bin_input_data_dir/dev --testpref $bin_input_data_dir/test \
 --destdir $bin_out_data_dir --workers $num_workers \
 --srcdict $download_dir/final_bin/dict.SRC.txt --tgtdict $download_dir/final_bin/dict.TGT.txt --thresholdtgt 5 --thresholdsrc 5

Running experiment /content/finetuning/dataset_finetune on mr to en
Applying normalization and script conversion for train 
100% 5168/5168 [00:00<00:00, 10012.02it/s]
100% 5168/5168 [00:01<00:00, 2825.64it/s]
Number of sentences in train : 5168
Applying normalization and script conversion for dev 
100% 1000/1000 [00:00<00:00, 4951.35it/s]
100% 1000/1000 [00:00<00:00, 1371.81it/s]
Number of sentences in dev : 1000
Applying normalization and script conversion for test 
100% 1000/1000 [00:00<00:00, 3896.86it/s]
100% 1000/1000 [00:00<00:00, 1386.31it/s]
Number of sentences in test : 1000

/content/finetuning/dataset_finetune/data/train.SRC
/content/finetuning/dataset_finetune/data/train.TGT
  0% 0/11 [00:00<?, ?it/s]src: as, tgt:en
src: bn, tgt:en
src: gu, tgt:en
src: hi, tgt:en
src: kn, tgt:en
src: ml, tgt:en
src: mr, tgt:en
/content/finetuning/dataset_finetune/norm/mr-en/train.mr
/content/finetuning/dataset_finetune/norm/mr-en/train.en
src: or, tgt:en
src: pa, tgt:en
src: ta, tgt:en
src:

In [ ]:
!( fairseq-train ../dataset_finetune/final_bin \
--max-source-positions=210 \
--max-target-positions=210 \
--max-update=1000 \
--save-interval=1 \
--arch=transformer_4x \
--criterion=label_smoothed_cross_entropy \
--source-lang=SRC \
--lr-scheduler=inverse_sqrt \
--target-lang=TGT \
--label-smoothing=0.1 \
--optimizer adam \
--adam-betas "(0.9, 0.98)" \
--clip-norm 1.0 \
--warmup-init-lr 1e-07 \
--warmup-updates 4000 \
--dropout 0.2 \
--tensorboard-logdir ../dataset_finetune/tensorboard-wandb \
--save-dir ../dataset_finetune/model \
--keep-last-epochs 5 \
--patience 5 \
--skip-invalid-size-inputs-valid-test \
--user-dir model_configs \
--update-freq=2 \
--distributed-world-size 1 \
--max-tokens 256 \
--lr 3e-5 \
--restore-file ../en-indic/model/checkpoint_best.pt \
--reset-lr-scheduler \
--reset-meters \
--reset-dataloader \
--reset-optimizer)

2023-05-04 07:26:20.852311: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-04 07:26:22 | INFO | numexpr.utils | NumExpr defaulting to 2 threads.
2023-05-04 07:26:27 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': '../dataset_finetune/tensorboard-wandb', 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': 'model_configs', 'empty_cache_freq'

In [ ]:
from finetuning.indicTrans.inference.engine import Model
indic2en_model_base = Model(expdir='/content/finetuning/indic-en')
indic2en_model_finetuned = Model(expdir='/content/finetuning/dataset_finetune')

Initializing vocab and bpe
Initializing model for translation
Initializing vocab and bpe
Initializing model for translation


In [ ]:
f=open("/content/finetuning/dataset_finetune/test/test.en","r")
test_en_list_full=[]
for x in f:
  test_en_list_full.append(x)

test_en_list=test_en_list_full

f=open("/content/finetuning/dataset_finetune/test/test.mr","r")
test_mr_list_full=[]
for x in f:
  test_mr_list_full.append(x)

test_mr_list=test_mr_list_full

In [ ]:
BATCH_SIZE = 2
NUM_BATCHES = len(test_mr_list)//BATCH_SIZE + 1
predictions_en_finetuned=[]
predictions_en_base=[]
for i in range(NUM_BATCHES):
  mr_sents_batch = test_mr_list[BATCH_SIZE*i : BATCH_SIZE*(i+1)]
  predictions_en_base.extend(indic2en_model_base.batch_translate(mr_sents_batch,'mr','en'))
  predictions_en_finetuned.extend(indic2en_model_finetuned.batch_translate(mr_sents_batch,'mr','en'))

100%|██████████| 2/2 [00:00<00:00, 2072.28it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


In [ ]:
ref_en_list=[[x] for x in test_en_list]

In [ ]:
from sacrebleu.metrics import BLEU
bleu = BLEU()

In [ ]:
bleu.corpus_score(predictions_en_base, ref_en_list)

BLEU = 44.18 100.0/57.1/33.3/20.0 (BP = 1.000 ratio = 1.000 hyp_len = 8 ref_len = 8)

In [ ]:
bleu.corpus_score(predictions_en_finetuned, ref_en_list)

BLEU = 65.90 83.3/72.7/70.0/44.4 (BP = 1.000 ratio = 1.000 hyp_len = 12 ref_len = 12)

In [ ]:
mr_sents=['चकावते ते सोने नसते','एक ना धड भाराभर चिंध्या','दुष्काळात तेरावा महिना','माकडाच्या हाती कोलीत','नाचता येईना अंगण वाकडे','अंथरूण पाहून पाय पसरावेत',
          'उथळ पाण्याला खळखळाट फार','बळी तो कान पिळी','एका हाताने टाळी वाजत नसते','इकडे आड तिकडे विहीर','वरातीमागून घोडे','अतिपरिचयात अवज्ञा',
          'वाहत्या गंगेत हात धुवून घेणे','बडा घर पोकळ वासा']
indic2en_model_finetuned.batch_translate(mr_sents, 'mr', 'en')

100%|██████████| 14/14 [00:00<00:00, 18844.75it/s]


['He said.',
 'The Prime Minister said that the India.',
 'The Prime Minister',
 'The Prime Minister, the Prime Minister of the',
 'The Prime Minister, the Prime Minister of the Prime Minister.',
 'The Prime Minister, the Prime Minister.',
 'The Prime Minister, the Prime Minister of the Prime Minister.',
 'The Prime Minister',
 'The Prime Minister, Shri Narendra Modi.',
 'The Prime Minister, Shri Narendra Modi.',
 'The Prime Minister',
 'The Prime Minister',
 'He said.',
 'The Prime Minister, the']

In [ ]:
mr_sents=['चकावते ते सोने नसते','एक ना धड भाराभर चिंध्या','दुष्काळात तेरावा महिना','माकडाच्या हाती कोलीत','नाचता येईना अंगण वाकडे','अंथरूण पाहून पाय पसरावेत',
          'उथळ पाण्याला खळखळाट फार','बळी तो कान पिळी','एका हाताने टाळी वाजत नसते','इकडे आड तिकडे विहीर','वरातीमागून घोडे','अतिपरिचयात अवज्ञा',
          'वाहत्या गंगेत हात धुवून घेणे','बडा घर पोकळ वासा']
indic2en_model_base.batch_translate(mr_sents, 'mr', 'en')

100%|██████████| 14/14 [00:00<00:00, 13394.22it/s]


['Gold is not a metal',
 'Not a single scratch',
 '13th month of drought',
 "In the monkey's hand",
 'Unable to dance',
 'Spread your feet by looking at the bed',
 'Very deep water',
 'The victim has a yellow ear',
 'No clapping with one hand',
 'A well here and there',
 'Horses behind the chariot',
 'Over-identity denial',
 'Washing hands with soap',
 'The big house']